In [ ]:
import numpy as np
from pyannote.core import Segment, Timeline
import os
from sklearn.cluster import KMeans
import pandas as pd
import json
import pyannote.core.json
from numpy import genfromtxt
from sklearn.cluster import KMeans
from pydub import AudioSegment
from pydub.playback import play
from pydub import AudioSegment

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Play/Timestamps' )
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
# import the complete file timestamps (from 0 to file end)
json_timelines=list()
for file in file_names:
    json_timelines.append(pyannote.core.json.load_from(file))

In [ ]:
# import the embedding vectors 
os.chdir( '/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Play/Embeddings' )
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])
embeddings=list()
for file in file_names:
    embeddings.append(genfromtxt(file, delimiter=','))

In [ ]:
# run k means 
kmeans = KMeans(n_clusters=4)
predictions = list()
for file in embeddings:       
    labels = kmeans.fit_predict(file)
    predictions.append(labels)

In [ ]:
# tuple of 1-s second and its corresponding label 
list_of_tuples=[]
temp_list=[]
for prediction, timeline in zip(predictions, json_timelines):
    for label, segment in zip(prediction, timeline):
        temp_list.append(((label,segment)))
    list_of_tuples.append(temp_list)
    temp_list=[]
    

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/overlapping_speech/no_os_play')  #if string starts with slash it is considered absolute
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
# make new timelines from clusters 
def timelines_from_clusters(list_of_tuples,cluster_no):
    timelines_from_clusters = []
    for file in list_of_tuples:
        temp_list=[]
        for cluster_seg in file:
            if cluster_seg[0]==cluster_no:
                temp_list.append(cluster_seg[1])
        timelines_from_clusters.append(temp_list)
        
    return timelines_from_clusters


In [ ]:
participants = [1043, 1049 ,1062, 1073 ,1080, 1082, 1089, 1097, 1098, 1104, 1108, 1117, 1118, 1131, 1161, 1180, 1184, 1188, 1210, 1234,
1245, 1263, 1282, 2027]


In [ ]:
locations = []
for x in file_names:
    if int(x.split("_")[0]) in participants:
        locations.append(file_names.index(x))
locations

In [ ]:
clusters_0 = timelines_from_clusters(list_of_tuples,0)
clusters_1 = timelines_from_clusters(list_of_tuples,1)
clusters_2 = timelines_from_clusters(list_of_tuples,2)
clusters_3 = timelines_from_clusters(list_of_tuples,3)


In [ ]:

locations

In [ ]:
for x in clusters_0:
    if clusters_0.index(x) in locations:
        print(x)


In [ ]:
# removing participants (child here) who have missing diarization data in play (i.e. all of their 4 clusters were noise)
clusters_0 = [x for x in clusters_0 if clusters_0.index(x) in locations]
clusters_1 = [x for x in clusters_1 if clusters_1.index(x) in locations]
clusters_2 = [x for x in clusters_2 if clusters_2.index(x) in locations]
clusters_3 = [x for x in clusters_3 if clusters_3.index(x) in locations]


In [ ]:
file_names = [file[0:4] for file in file_names]

file_names = [int(x) for x in file_names if file_names.index(x) in locations]

In [ ]:
clusters_0[3] =  clusters_0[3] + clusters_2[3]
clusters_2[10] = clusters_2[10] + clusters_3[10]
clusters_2[12] = clusters_2[12] + clusters_3[12]





In [ ]:
len(clusters_0)

In [ ]:
len(cg_0)

## now choose only the cg clusters 

In [ ]:
cg_0 = [x for x in clusters_0]
cg_0

In [ ]:
cg_1 = [x for x in clusters_1]

In [ ]:
len(clusters_2)

In [ ]:
clusters_2.pop(3)
cg_2 = [x for x in clusters_2]
len(cg_2)

In [ ]:
clusters_3.pop(12)
clusters_3.pop(10)
cg_3 = [x for x in clusters_3]
len(cg_3)

In [ ]:
index_cg_0 = [1062, 1073, 1082, 1104,  ]

In [ ]:
listt = index_cg_0 + index_cg_1 + index_cg_2 + index_cg_3
len(listt)


In [ ]:
len(index_child_0), len(cg_0)

In [ ]:
index_cg_1 = [participants]

In [ ]:
len(index_child_1), len(cg_1)

In [ ]:
index_cg_2 = [1091, 1107, 1108, 1112, 1132]

In [ ]:
len(index_child_2), len(cg_2)

In [ ]:
index_cg_3 = [1047, 1079, 1092, 1096, 1099, 1170, 1180]

In [ ]:
len(index_child_3), len(cg_3)

In [ ]:
cg_speech_0 = []
for index, caregiver in zip(index_cg_0, cg_0,):
    cg_speech_0.append((index,caregiver))
cg_speech_0

In [ ]:
cg_speech_1 = []
for index, caregiver in zip(index_cg_1, cg_1,):
    cg_speech_1.append((index,caregiver))
cg_speech_1

In [ ]:
cg_speech_2 = []
for index, caregiver in zip(index_cg_2, cg_2):
    cg_speech_2.append((index,caregiver))
cg_speech_2

In [ ]:
 cg_speech_3 = []
for index, caregiver in zip(index_cg_3, cg_3):
    cg_speech_3.append((index,caregiver))
cg_speech_3

In [ ]:
len(cg_speech_0) + len(cg_speech_1) + len(cg_speech_2) + len(cg_speech_3)

In [ ]:
cg_segments = sorted(cg_speech_0 + cg_speech_1 + cg_speech_2 + cg_speech_3)

In [ ]:
timelines_list = []


for index, segments in cg_segments:
    segment_holder =[]
    timeline = Timeline(segments=[s for s in segments], uri = str(index))
    timelines_list.append(timeline)
        
    

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Meal/identification/speech_turns_child')  #if string starts with slash it is considered absolute


In [ ]:
for timeline in timelines_list:
    pyannote.core.json.dump_to(timeline, timeline.uri+".json")